In [134]:
import pandas as pd
import subprocess
import os
import requests
from bs4 import BeautifulSoup
import json
import git
import plotly.express as px
import numpy as np
from datetime import datetime
import missingno as msno

In [172]:
csv_url="https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

In [174]:
df = pd.read_csv(csv_url)#,sep=',')#, error_bad_lines=False)

In [176]:
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196540,ZWE,Africa,Zimbabwe,2022-06-20,254831.0,30.0,96.571,5534.0,0.0,1.857,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
196541,ZWE,Africa,Zimbabwe,2022-06-21,254919.0,88.0,109.143,5536.0,2.0,2.143,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
196542,ZWE,Africa,Zimbabwe,2022-06-22,255049.0,130.0,94.571,5541.0,5.0,2.286,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
196543,ZWE,Africa,Zimbabwe,2022-06-23,255136.0,87.0,90.571,5542.0,1.0,2.286,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


In [182]:
df_GER=df['location']=='Germany'
df_IND=df['location']=='India'
df_USA=df['location']=='United States'

In [183]:
df_all= df_GER | df_IND | df_USA 

In [184]:
df_list=df[df_all]

In [185]:
df_list

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
66278,DEU,Europe,Germany,2020-01-27,1.0,1.0,NaN,NaN,NaN,NaN,...,28.2,33.1,NaN,8.00,81.33,0.947,NaN,NaN,NaN,NaN
66279,DEU,Europe,Germany,2020-01-28,4.0,3.0,NaN,NaN,NaN,NaN,...,28.2,33.1,NaN,8.00,81.33,0.947,NaN,NaN,NaN,NaN
66280,DEU,Europe,Germany,2020-01-29,4.0,0.0,NaN,NaN,NaN,NaN,...,28.2,33.1,NaN,8.00,81.33,0.947,NaN,NaN,NaN,NaN
66281,DEU,Europe,Germany,2020-01-30,4.0,0.0,NaN,NaN,NaN,NaN,...,28.2,33.1,NaN,8.00,81.33,0.947,NaN,NaN,NaN,NaN
66282,DEU,Europe,Germany,2020-01-31,5.0,1.0,NaN,NaN,NaN,NaN,...,28.2,33.1,NaN,8.00,81.33,0.947,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186059,USA,North America,United States,2022-06-20,86297081.0,50980.0,90015.571,1013493.0,80.0,278.571,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN
186060,USA,North America,United States,2022-06-21,86452232.0,155151.0,98999.286,1013975.0,482.0,292.857,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN
186061,USA,North America,United States,2022-06-22,86636306.0,184074.0,99152.714,1014835.0,860.0,318.286,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN
186062,USA,North America,United States,2022-06-23,86757621.0,121315.0,99913.286,1015342.0,507.0,329.714,...,19.1,24.6,NaN,2.77,78.86,0.926,NaN,NaN,NaN,NaN


In [186]:
df_list.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [187]:
fig=px.line(df_list, x= "date", y= "total_cases",color='location', title= "Covid Cases")
fig.show()

In [188]:
df_list['total_infection_mean']=df_list['total_cases']/df_list['population']

C:\Users\SLP279\AppData\Local\Temp\ipykernel_15032\1847970003.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [189]:
fig=px.line(df_list, x= "date", y= "total_infection_mean",color='location', title= "Covid Cases")
fig.show()

In [190]:
df_list['total_vaccination_mean']=df_list['people_fully_vaccinated']/df_list['population']

C:\Users\SLP279\AppData\Local\Temp\ipykernel_15032\3588373942.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [191]:
fig=px.line(df_list, x= "date", y= "total_vaccination_mean",color='location', title= "Covid Cases")
fig.show()